# Подготовим данные для обучении модели,заполним пропуски и создадим новые фичи

## Импорт библиотек

In [53]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np

### Считывание данных 

In [54]:
df = pd.read_csv('vehicles.csv')

In [55]:
data = df.copy()
df_train,df_test = train_test_split(data,test_size=0.2,random_state=42)

## Повторим все манипуляции над данным из файла eda.ipynb,а также поэкспрементируем с новыми фичами

## Переведем колонки cylinders и posting_date в другие форматы,для более удобной работы с ними в дальнейшем

In [56]:
# Преобразование цилиндров
df_train['cylinders_num'] =  df_train['cylinders'].str.extract('(\d+)')
df_train['cylinders_num'] = df_train['cylinders_num'].astype(float)

# Создание новых числовых признаков
df_train['posting_date'] = pd.to_datetime(df_train['posting_date'], errors='coerce', utc=True)
df_train['posting_year'] = df_train['posting_date'].dt.year
df_train['posting_month'] = df_train['posting_date'].dt.month
df_train['posting_day'] = df_train['posting_date'].dt.day
df_train['posting_weekday'] = df_train['posting_date'].dt.weekday  
df_train['posting_hour'] = df_train['posting_date'].dt.hour

df_train.drop(columns=['posting_date','posting_year'],inplace=True)


df_test['cylinders_num'] = df_test['cylinders'].str.extract('(\d+)')
df_test['cylinders_num'] = df_test['cylinders_num'].astype(float)


df_test['posting_date'] = pd.to_datetime(df_test['posting_date'], errors='coerce', utc=True)
df_test['posting_year'] = df_test['posting_date'].dt.year
df_test['posting_month'] = df_test['posting_date'].dt.month
df_test['posting_day'] = df_test['posting_date'].dt.day
df_test['posting_weekday'] = df_test['posting_date'].dt.weekday
df_test['posting_hour'] = df_test['posting_date'].dt.hour

# Удалим год публикации для того,чтобы 
df_test.drop(columns=['posting_date','posting_year'], inplace=True)

## Удалим колонки с большим количеством уникальных значений

In [57]:
df_train.drop(columns=['id', 'url', 'image_url', 'region_url','VIN'],inplace=True)
df_test.drop(columns=['id', 'url', 'image_url', 'region_url','VIN'],inplace=True)

##  Удалим неинформативные столбцы  из обучающей выборки

In [58]:
def find_col_to_del(data:pd.DataFrame):
    all_k = data.shape[0]
    cols_with_null = data.columns[data.isnull().sum() > 0].tolist()
    cols_to_drop = []
    for col in cols_with_null:
        if (data[col].isnull().sum() / all_k) * 100 > 70:
            cols_to_drop.append(col)
    return cols_to_drop

cols_to_drop = find_col_to_del(data=df_train)
df_train.drop(columns=cols_to_drop,inplace=True)
df_test.drop(columns=cols_to_drop,inplace=True)

## Аналогично поступим и со строками

In [59]:
df_train = df_train.dropna(thresh=int((df_train.shape[1] / 100) * 70))

## Заполним пропуски

#### Числовые 

In [60]:
def find_num_col_to_fill(data: pd.DataFrame):
    num_cols = data.select_dtypes(exclude=['object'])
    num_cols_with_null = num_cols.columns[num_cols.isnull().any()].tolist()
    return  num_cols_with_null

num_cols_to_fil = find_num_col_to_fill(df_train)

df_train[num_cols_to_fil] = df_train[num_cols_to_fil].fillna(df_train[num_cols_to_fil].mean())
df_test[num_cols_to_fil] = df_test[num_cols_to_fil].fillna(df_train[num_cols_to_fil].mean())

#### Категориальные 

In [61]:
def find_cot_col_to_fill(data: pd.DataFrame):
    cot_cols = data.select_dtypes(include=['object'])
    cot_cols_with_null = [col for col in cot_cols.columns if cot_cols[col].isnull().any() and col != 'description']
    return cot_cols_with_null

cot_cols_to_fil = find_cot_col_to_fill(df_train)
df_train[cot_cols_to_fil] = df_train[cot_cols_to_fil].fillna('unknown')
df_train['description'] = df_train['description'].fillna('')
df_test[cot_cols_to_fil] = df_test[cot_cols_to_fil].fillna('unknown')
df_test['description'] = df_test['description'].fillna('')

## Очистим от выбросов

### Для начала уберем явные выбросы

In [62]:
df_train = df_train[(df_train['price'] <= 500_000) & (df_train['price'] >= 100)]
df_train = df_train[df_train['odometer'] <= 600_000]

###  Для более объективного и воспроизводимого отбора выбросов я применю метод IQR с кастомными квантилями. Это позволит формально выделить аномалии и гибко настроить границы через коэффициент k, избегая чрезмерного удаления данных.

In [63]:
def delete_by_IQR(data: pd.DataFrame, column: str, q_low: float = 0.25, q_high: float = 0.75) -> None:
    if column not in data.columns:
        print(f"Колонка '{column}' не найдена в DataFrame")
    
    else:
        Q1 = data[column].quantile(q_low)
        Q3 = data[column].quantile(q_high)
        IQR = Q3 - Q1

        low_border = Q1 - 1.5 * IQR
        high_border = Q3 + 1.5 * IQR

        before = len(data)
        data.drop(data[(data[column] < low_border) | (data[column] > high_border)].index, inplace=True)
        after = len(data)
        print(f"'{column}': удалено {before - after} выбросов")


df_train_clean = df_train.copy()
for col in ['year', 'price', 'odometer','long','lat']:
    delete_by_IQR(df_train_clean, col, q_low=0.1, q_high=0.90)

'year': удалено 5943 выбросов
'price': удалено 676 выбросов
'odometer': удалено 250 выбросов
'long': удалено 18 выбросов
'lat': удалено 46 выбросов


### Посторим на размер датасета после всех удалений

In [64]:
print('Размер датасета: ',df_train_clean.shape)

Размер датасета:  (304255, 23)


## Выделим наиболее встречающиеся значения в колнках region и model 

### Region

In [65]:
region_counts = df_train_clean['region'].value_counts()
top100 = region_counts.head(100)
df_train_clean['region_top100'] = df_train_clean['region'].apply(lambda x: x if x in top100.index else 'Other')
df_test['region_top100'] = df_test['region'].apply(lambda x: x if x in top100.index else 'Other')

### Model

In [66]:
model_counts = df_train_clean['model'].value_counts()
top250 = model_counts.head(250)
df_train_clean['model_top250'] = df_train_clean['model'].apply(lambda x: x if x in top250.index else 'Other')
df_test['model_top250'] = df_test['model'].apply(lambda x: x if x in top250.index else 'Other')

In [67]:
df_train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304255 entries, 366318 to 121958
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   region           304255 non-null  object 
 1   price            304255 non-null  int64  
 2   year             304255 non-null  float64
 3   manufacturer     304255 non-null  object 
 4   model            304255 non-null  object 
 5   condition        304255 non-null  object 
 6   cylinders        304255 non-null  object 
 7   fuel             304255 non-null  object 
 8   odometer         304255 non-null  float64
 9   title_status     304255 non-null  object 
 10  transmission     304255 non-null  object 
 11  drive            304255 non-null  object 
 12  type             304255 non-null  object 
 13  paint_color      304255 non-null  object 
 14  description      304255 non-null  object 
 15  state            304255 non-null  object 
 16  lat              304255 non-null  floa

In [68]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85376 entries, 100905 to 374513
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   region           85376 non-null  object 
 1   price            85376 non-null  int64  
 2   year             85376 non-null  float64
 3   manufacturer     85376 non-null  object 
 4   model            85376 non-null  object 
 5   condition        85376 non-null  object 
 6   cylinders        85376 non-null  object 
 7   fuel             85376 non-null  object 
 8   odometer         85376 non-null  float64
 9   title_status     85376 non-null  object 
 10  transmission     85376 non-null  object 
 11  drive            85376 non-null  object 
 12  type             85376 non-null  object 
 13  paint_color      85376 non-null  object 
 14  description      85376 non-null  object 
 15  state            85376 non-null  object 
 16  lat              85376 non-null  float64
 17  long       

## Добавим новые признаки

### Переписать 

In [ ]:
def create_features(data,num_columns = []):
    """Функция для создания новых фич"""

    ### Для начала логарифмировуем  
    for col in num_columns:
        if col not in ['price','lat','long']:
            data[f'log_{col}'] = np.log1p(data[col])
    
    ### Создадим новые признаки
    data['car_age'] = 2021 - data['year']
    data['mileage_per_year'] = data['odometer'] / (data['car_age'] + 1)

    ### Сделаем бины по координатам
    data['region_lat_bin'] = pd.cut(data['lat'], bins=10, labels=False)
    data['region_long_bin'] = pd.cut(data['long'], bins=10, labels=False)
    data['lat_long_cluster'] = (data['region_lat_bin'] * 10 + data['region_long_bin']).astype(int)

    ### Сделаем признаки на основе описания (можно добавить ещё) 
    data['len_description'] = data['description'].str.len()
    data['word_count'] = data['description'].str.split().apply(len)


In [70]:
num_cols = df_train_clean.select_dtypes(exclude='object').columns
create_features(df_train_clean,num_cols)
create_features(df_test,num_cols)

/opt/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [73]:
df_train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304255 entries, 366318 to 121958
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   region               304255 non-null  object 
 1   price                304255 non-null  int64  
 2   year                 304255 non-null  float64
 3   manufacturer         304255 non-null  object 
 4   model                304255 non-null  object 
 5   condition            304255 non-null  object 
 6   cylinders            304255 non-null  object 
 7   fuel                 304255 non-null  object 
 8   odometer             304255 non-null  float64
 9   title_status         304255 non-null  object 
 10  transmission         304255 non-null  object 
 11  drive                304255 non-null  object 
 12  type                 304255 non-null  object 
 13  paint_color          304255 non-null  object 
 14  description          304255 non-null  object 
 15  state            

## Также более детально поработаем с текстом,а именно с описанием

### Применим TF-IDF для кодировки категориальных признаков

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [72]:
# # 1. Создаём TF-IDF и задаём параметры
# tfidf = TfidfVectorizer(
#     max_features=2000,    # оставляем только топ слов
#     stop_words='english', # английские стоп-слова
#     ngram_range=(1,2)     # униграммы и биграммы
# )

# # 2. Учим TF-IDF на train
# X_train_desc = tfidf.fit_transform(df_train['description'].fillna(''))

# # 3. Применяем тот же TF-IDF к valid/test
# X_valid_desc = tfidf.transform(df_test['description'].fillna(''))

# # 4. Объединяем с числовыми признаками
# X_train_final = hstack([df_train.drop(columns=['description']), X_train_desc])
# X_valid_final = hstack([df_test.drop(columns=['description']), X_valid_desc])

# Проверка 

### Сравнивая метрики моделей, обученных на baseline, можно оценить эффективность внесённых изменений в признаки и модель.